In [49]:
!pip install inflect --upgrade
!pip install contractions
!pip install -U gensim
import gensim
from gensim.models.word2vec import Word2Vec
import pandas as pd
import numpy as np
import html
import re, string, unicodedata
import nltk
import inflect
import contractions
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

Requirement already up-to-date: inflect in /usr/local/lib/python3.6/dist-packages (2.1.0)
Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.7.1)


# Part 1 - Working with Text Data

### Use Python string methods remove irregular whitespace from the following string:

In [4]:
whitespace_string = "\n\n  This is a    string   that has  \n a lot of  extra \n   whitespace.   "
print(whitespace_string)




  This is a    string   that has  
 a lot of  extra 
   whitespace.   


In [5]:
 ##### Your Code Here #####
import re
whitespace_string = whitespace_string.replace('\n', ' ') # Use to remove carriage returns. 
whitespace_string = re.sub(' +', ' ', whitespace_string) # Remove excess whitespace. 
print(whitespace_string)

 This is a string that has a lot of extra whitespace. 


### Use Regular Expressions to take the dates in the following .txt file and put them into a dataframe with columns for:

[RegEx dates.txt](https://github.com/ryanleeallred/datasets/blob/master/dates.txt)

- Day
- Month
- Year


In [6]:
##### Your Code Here #####
url = "https://raw.githubusercontent.com/ryanleeallred/datasets/master/dates.txt"
df = pd.read_csv(url, names = ["Month", "Day", "Year"], sep = "(?:,|\s)\s*",
                 header = None,  engine='python')
df = df[["Day","Month","Year"]]
df

,Day,Month,Year
0,8,March,2015
1,15,March,2015
2,22,March,2015
3,29,March,2015
4,5,April,2015
5,12,April,2015
6,19,April,2015
7,26,April,2015
8,3,May,2015
9,10,May,2015


# Part 2 - Bag of Words 

### Use the twitter sentiment analysis dataset found at this link for the remainder of the Sprint Challenge:

[Twitter Sentiment Analysis Dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv)

 ### Clean and tokenize the documents ensuring the following properties of the text:

1) Text should be lowercase.

2) Stopwords should be removed.

3) Punctuation should be removed.

4) Tweets should be tokenized at the word level. 

(The above don't necessarily need to be completed in that specific order.)

### Output some cleaned tweets so that we can see that you made all of the above changes.


In [7]:
df = pd.read_csv("https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv")
df.head()

,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...


In [32]:
df = pd.read_csv("https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv")
df.head()

def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    return df
  
df = standardize_text(df, 'SentimentText')


### Denoise functions, apply to text ### 
def encode_decode(text):
    """Utility function to clean text by decoding HTML text."""
    unescaped = html.unescape(text)
    return unescaped

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
  

def denoise_text(text):
    text = encode_decode(text)
    text = remove_between_square_brackets(text)
    return text


df['SentimentText'] = df['SentimentText'].apply(denoise_text)


### Normalizing functions, apply to tokenized lists ###
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words or string"""
    new_words = []
    for word in words:
        new_word = (unicodedata.normalize('NFKD', word)
                    .encode('ascii','ignore').decode('utf-8', 'ignore'))
        new_words.append(new_word)
    return new_words

  
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words or string, also 
    lowercases every word.
    """
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s\_]|\_', '', word)
        new_word = new_word.lower()
        if new_word != '':
            new_words.append(new_word)
    return new_words

  
def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words or string 
    with textual representation.
    """
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words  
  
  
def normalize(words):
    words = remove_non_ascii(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    #words = remove_stopwords(words)
    return words
  
#def remove_stopwords(words):
#    """Remove stop words from list of tokenized words"""
#    new_words = []
#    for word in words:
#        if word not in stopwords:
#            new_words.append(word)
#    return new_words
  
  
tokenizer = RegexpTokenizer(r'\w+')
df['SentimentText'] = df['SentimentText'].apply(tokenizer.tokenize)
df['SentimentText'] = [normalize(token) for token in df['SentimentText']]


cleaned_words = [word for tokens in df['SentimentText'] for word in tokens]
cleanedVOCAB = sorted(list(set(cleaned_words)))
display("%s words total, with a vocabulary size of %s" % (len(cleaned_words), len(cleanedVOCAB)))

display(df.head(20))

'1277018 words total, with a vocabulary size of 52722'

,Sentiment,SentimentText
0,0,"[is, so, sad, for, my, apl, friend]"
1,0,"[i, missed, the, new, moon, trailer]"
2,1,"[omg, its, already, seven, thirty, o]"
3,0,"[omgaga, im, sooo, im, gunna, cry, i, ve, been..."
4,0,"[i, think, mi, bf, is, cheating, on, me, tt]"
5,0,"[or, i, just, worry, too, much]"
6,1,"[juuuuuuuuuuuuuuuuussssst, chillin]"
7,0,"[sunny, again, work, tomorrow, tv, tonight]"
8,1,"[handed, in, my, uniform, today, i, miss, you,..."
9,1,"[hmmmm, i, wonder, how, she, my, number]"


### How should TF-IDF scores be interpreted? How are they calculated?

TF-IDF stands for **Term Frequency - Inverse Document Frequency**

**Term Frequency** shows how frequently an expression (term, word) occurs in a document, with the insight being that the more frequently a term is used, the greater its importance.  It is calculated by taking the number of times a word appears in a document, divided by the total number of words in that document

In sklearn TF-IDF Transformer it is caluclated by: ***tf(t, d)***

It is well understood however the correlation between Term Frequency != Importance in all cases. 
We see terms that occur so often they necessitate removal via stopwords removal, as they are trivial but frequent. Hence frequency alone is not conclusive. 

**Inverse Document Frequency**  It is a method of determining the frequency of a word within a dataset of texts. Imagine it as a ratio of all existing texts/documents of an entire dataset and the number of texts that contain the defined keyword.  

In sklearn TF-IDF Transformer it is caluclated by: ***idf(t) = log [ n / df(t) ] + 1***

n = total_#_of_documents
df(t) #_of_docs_containing_term;  the document frequency of t;  the number of documents in the document set that contain the term t. 

As the document Frequency grows this fraction (total/target) becomes smaller.

**TF-IDF** or more accurately (*TF* x *IDF*) is where we multiply the Term Frequency by the Inverse Document Frequency.  We essentially weight the Term Frequency by the Inverse Document Frequency so that words that occur too frequently are penalized, and words that may be more infrequent but appear across multiple documents are boosted.  This better helps sharpen the signal of our input data and allows our models to operate with a clearer distinction between signal (important words) & noise (frequent but inconsequential words). 




# Part 3 - Document Classification

1) Use Train_Test_Split to create train and test datasets.

2) Vectorize the tokenized documents using your choice of vectorization method. 

 - Stretch goal: Use both of the methods that we talked about in class.

3) Create a vocabulary using the X_train dataset and transform both your X_train and X_test data using that vocabulary.

4) Use your choice of binary classification algorithm to train and evaluate your model's accuracy. Report both train and test accuracies.

 - Stretch goal: Use an error metric other than accuracy and implement/evaluate multiple classifiers.



## Bag of Words w/ Count Vectorizer

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
seed = 42

In [0]:
# Defining the Count Vectorizer function. 
def cv(data):
    count_vectorizer = CountVectorizer()
    emb = count_vectorizer.fit_transform(data)
    return emb, count_vectorizer

# Here I define my corpus and lables. These are used throughout all the other models/processes.
list_corpus = df["SentimentText"].apply(lambda y: " ".join(str(x) for x in y)).tolist()
list_labels = df["Sentiment"].tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
                                                                                random_state=seed)

X_train_counts, count_vectorizer = cv(X_train)
X_test_counts = count_vectorizer.transform(X_test)

In [0]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=40)
clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)

In [40]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1

accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.734, precision = 0.735, recall = 0.734, f1 = 0.735


In [47]:
from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB().fit(X_train_counts, y_train)

train_predictions = MNB.predict(X_train_counts)
test_predictions = MNB.predict(X_test_counts)

accuracy, precision, recall, f1 = get_metrics(y_test, test_predictions)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.758, precision = 0.758, recall = 0.758, f1 = 0.758


## Now BoW TF-IDF Vectorizer

In [0]:
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()

    train = tfidf_vectorizer.fit_transform(data)

    return train, tfidf_vectorizer

X_train_tfidf, tfidf_vectorizer = tfidf(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [0]:
clf_tfidf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
clf_tfidf.fit(X_train_tfidf, y_train)

y_predicted_tfidf = clf_tfidf.predict(X_test_tfidf)

In [44]:
accuracy_tfidf, precision_tfidf, recall_tfidf, f1_tfidf = get_metrics(y_test, y_predicted_tfidf)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy_tfidf, precision_tfidf, 
                                                                       recall_tfidf, f1_tfidf))

accuracy = 0.743, precision = 0.744, recall = 0.743, f1 = 0.743


# Part 4 -  Word2Vec

1) Fit a Word2Vec model on your cleaned/tokenized twitter dataset. 

2) Display the 10 words that are most similar to the word "twitter"

## Word2Vec

In [0]:
model = Word2Vec(df["SentimentText"].tolist(), min_count=1, size=5)

In [51]:
print(model)
print(list(model.wv.vocab))
print(len(model.wv.vocab))

Word2Vec(vocab=52722, size=5, alpha=0.025)
['is', 'so', 'sad', 'for', 'my', 'apl', 'friend', 'i', 'missed', 'the', 'new', 'moon', 'trailer', 'omg', 'its', 'already', 'seven', 'thirty', 'o', 'omgaga', 'im', 'sooo', 'gunna', 'cry', 've', 'been', 'at', 'this', 'dentist', 'since', 'eleven', 'was', 'suposed', 'two', 'just', 'get', 'a', 'crown', 'put', 'on', '30mins', 'think', 'mi', 'bf', 'cheating', 'me', 'tt', 'or', 'worry', 'too', 'much', 'juuuuuuuuuuuuuuuuussssst', 'chillin', 'sunny', 'again', 'work', 'tomorrow', 'tv', 'tonight', 'handed', 'in', 'uniform', 'today', 'miss', 'you', 'hmmmm', 'wonder', 'how', 'she', 'number', 'must', 'about', 'positive', 'thanks', 'to', 'all', 'haters', 'up', 'face', 'day', 'one hundred and twelve', 'one hundred and two', 'weekend', 'has', 'sucked', 'far', 'jb', 'isnt', 'showing', 'australia', 'any', 'more', 'ok', 'thats', 'it', 'win', 'way', 'feel', 'right', 'now', 'awhhe', 'man', 'm', 'completely', 'useless', 'rt', 'funny', 'can', 'do', 'twitter', 'feeling

In [54]:
model.wv.most_similar('twitter')

[('faber', 0.9973717331886292),
 ('mandarin', 0.9968427419662476),
 ('ncavp', 0.996773898601532),
 ('configuration', 0.9949542284011841),
 ('aux', 0.9943576455116272),
 ('corp', 0.9942663908004761),
 ('sanju', 0.9940757751464844),
 ('helloooo', 0.9936681389808655),
 ('jaded', 0.9935271739959717),
 ('fakt', 0.9930754899978638)]